# 📝 Homework 2: Data Wrangling - From Business Question to Analysis

<a href="https://colab.research.google.com/github/bradleyboehmke/uc-bana-4080/blob/main/homework/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## 📂 Instructions

This homework is based on the Lab we worked through in Thursday's class.  So, if you completed that Lab, you can use that notebook for the homework.

Complete the tasks below in this Jupyter notebook. Most tasks require you to write Python code and use the output to answer **a separate online quiz**.

At the end, you’ll also upload this completed `.ipynb` notebook

---

In this homework, we’ll use **three datasets** from the Complete Journey retail grocery data:

1. **transactions** – product purchases by households (receipt-level detail)  
2. **demographics** – household-level demographic data  
3. **products** – metadata about products purchased  

This homework reinforces this week’s readings:

- **[Reading 10: Manipulating Data](https://bradleyboehmke.github.io/uc-bana-4080/10-manipulating-data.html)**
- **[Reading 11: Summarizing Data](https://bradleyboehmke.github.io/uc-bana-4080/11_aggregating_data.html)**
- **[Reading 12: Joining Data](https://bradleyboehmke.github.io/uc-bana-4080/12-joining-data.html)**

We will:
- Start with simple data exploration
- Progress to manipulating and summarizing data
- End with joining datasets to answer more complex questions
- Practice breaking business questions into **analytical steps**

You are encouraged to work in small groups of **2–4 students** but you must submit your own notebook.


## Setup

In [4]:
# If you don't have completejourney_py installed, run:
!pip install completejourney-py
from completejourney_py import get_data
import pandas as pd

# Load datasets
cj_data = get_data()
transactions = cj_data['transactions']
products = cj_data['products']
demographics = cj_data['demographics']

# Quick preview
transactions.head()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 55.4 MB/s eta 0:00:00


,household_id,store_id,basket_id,product_id,quantity,sales_value,retail_disc,coupon_disc,coupon_match_disc,week,transaction_timestamp
0,900,330,31198570044,1095275,1,0.50,0.00,0.0,0.0,1,2017-01-01 11:53:26
1,900,330,31198570047,9878513,1,0.99,0.10,0.0,0.0,1,2017-01-01 12:10:28
2,1228,406,31198655051,1041453,1,1.43,0.15,0.0,0.0,1,2017-01-01 12:26:30
3,906,319,31198705046,1020156,1,1.50,0.29,0.0,0.0,1,2017-01-01 12:30:27
4,906,319,31198705046,1053875,2,2.78,0.80,0.0,0.0,1,2017-01-01 12:30:27


## Part 1 – Basic Exploration


**Q0:** How many transactions are in our dataset, what is the date range, how many households have demographic data, how many products exist, and what are the min/max/mean sales values?  

**Step-by-step instructions:**
1. Use `.shape[0]` on `transactions` to count rows.  
2. Use `.min()` and `.max()` on `transaction_timestamp` to find the date range.  
3. Use `.shape[0]` on `demographics` and `products` to get counts.  
4. Use `.min()`, `.max()`, `.mean()` on `sales_value` for basic stats.


In [18]:
# Starter code with blanks to fill
# total number of transactions
num_transactions = transactions.shape[0]
print(f"Number of transactions = {num_transactions}")

Number of transactions = 1469307


In [19]:
# date range of transactions
min_date = transactions['transaction_timestamp'].min()
max_date = transactions['transaction_timestamp'].max()
print(f"Minimum date = {min_date}, Maximum date = {max_date}")

Minimum date = 2017-01-01 11:53:26, Maximum date = 2018-01-01 04:01:20


In [22]:
# number of unique households and products
num_households = demographics.shape[0]
num_products = products.shape[0]
print(f"Number of houshold = {num_households}, Number of product = {num_products}")

Number of houshold = 801, Number of product = 92331


In [20]:
# summary statistics for sales_value
min_sales = transactions['sales_value'].min()
max_sales = transactions['sales_value'].max()
mean_sales = transactions['sales_value'].mean()
print(f"Minimum sale = {min_sales}, Maximum sales = {max_sales}, Mean sales = {mean_sales}")

Minimum sale = 0.0, Maximum sales = 840.0, Mean sales = 3.12803218115751



**Q1:** Which day had the highest total sales?  

**Step-by-step instructions:**
1. Create a new column `date` by extracting only the date from `transaction_timestamp` (`.dt.date`).  
2. Group by `date` and sum `sales_value`.  
3. Sort results in descending order.  
4. Select the top row.


In [11]:
# Your code here

# extracting date
transactions['date'] = transactions['transaction_timestamp'].dt.date

# group by date and sum sales
daily_sales = (transactions.groupby('date')['sales_value'].sum().sort_values(ascending=False))

daily_sales.head(1)

,sales_value
date,
2017-12-23,24994.47



**Q2:** What are the top 5 departments by total sales?  

**Step-by-step instructions:**
1. Join `transactions` to `products` on `product_id` using an inner join.  
2. Group by `department` and sum `sales_value`.  
3. Sort results in descending order.  
4. Display the top 5.


In [12]:
# Your code here

# join transactions with productions
trans_prod = transactions.merge(products, on='product_id', how='inner')

# group by department and sum sales
dept_sales = (trans_prod.groupby('department')['sales_value'].sum().sort_values(ascending=False))

dept_sales.head(5)

,sales_value
department,
GROCERY,2316393.89
DRUG GM,596827.45
FUEL,329594.45
PRODUCE,322858.82
MEAT,308575.33


## Part 2 – Manipulating Data


**Q3:** What is the average unit price for each department?  

**Step-by-step instructions:**
1. Create a `unit_price` column: `sales_value / quantity`.  
2. Join `transactions` to `products` to bring in `department`.  
3. Group by `department` and calculate the mean of `unit_price`.


In [13]:
# Your code here

# create unit price column
transactions['unit_price'] = transactions['sales_value'] / transactions['quantity']

# joining products to get department
trans_prod = transactions.merge(products, on='product_id', how='inner')

# mean of unit price by department
unit_price_by_dept = (trans_prod.groupby('department')['unit_price'].mean())

unit_price_by_dept

,unit_price
department,
AUTOMOTIVE,7.216111
CHEF SHOPPE,2.522274
CNTRL/STORE SUP,3.150000
COSMETICS,4.138923
COUPON,1.296070
DELI,inf
DRUG GM,inf
ELECT &PLUMBING,1.000000
FLORAL,7.732635



**Q4:** Do we have missing values in `unit_price`?  

**Step-by-step instructions:**
1. Use `.isna().sum()` on `unit_price` to count missing values.  
2. Consider filtering rows where `quantity == 0` to see if that’s the cause.


In [21]:
# Your code here

# count missing unit price values
missing_unit_price = transactions['unit_price'].isna().sum()

print(f"Missing unit price = {missing_unit_price}")

# checking if missing values are caused by quantity = 0
print(f"Transactions with quantity 0 = {transactions[transactions['quantity']== 0].shape[0]}")

Missing unit price = 8820
Transactions with quantity 0 = 8869


## Part 3 – Aggregations


**Q5:** Which income level spends the most on average?

*Hint:* Join transactions to demographics, group by income, calculate mean sales per income level.


In [ ]:
# Your code here



**Q6:** Do households with kids spend more (on average) than households without kids?  

*Hint:* Use `kid_count` to group households by creating a new column (e.g., `has_kids`) that identifies whether a household has kids (`kid_count > 0`) or not (`kid_count == 0`). Note that the tricky part of this step is that `kid_count` is not a numeric variable 🤔. Compute the average spend for those with kids and those without.


In [ ]:
# Your code here



**Q7:** What are the top 5 departments by total quantity of items sold?  

*Hint:* Join to products, group by department, sum quantity, and sort.


In [ ]:
# Your code here


## Part 4 – Joins for Deeper Insights


**Q8:** Which product is purchased most frequently?  

*Hint:* Group by `product_id`, sum quantity, then join to products for description.


In [ ]:
# Your code here



**Q9:** Identify all products with “pizza” in `product_type` and find the one with the greatest total sales.  

*Hint:* Filter products where product_type contains "pizza" with `.str.contains("pizza", case=False, na=False)`, join to transactions, sum sales by product.


In [ ]:
# Your code here



**Q10:** Which product category brings in the most revenue for the highest-income households with kids?

*Hint:* Filter demographics for the highest income level & `kid_count > 0`, join to transactions and products, group by category and compute the sum of sales value.


In [ ]:
# Your code here



**Q11:** Which manufacturer has the highest total sales, and which department do they primarily sell in?  

*Hint:* Join transactions to products, group by manufacturer, sum sales, find top. Then, filter products for that top manufacturer and check which department(s) they are associated with.


In [ ]:
# Your code here



**Q12:** For each income level, what is the most frequently purchased product category?  

*Hint:* Join demographics → transactions → products, group by income & category, count quantity, get top per income.


In [ ]:
# Your code here


## Homework Deliverable


- Implement the code to answer the above questions.  
- Once you have all your answers, go to the homework quiz on Canvas and submit your answers.  
- Save your notebook — you'll upload it on Canvas as part of the homework.
